# 使用 Tensorflow 识别 MNIST

## 1. 读入数据

MNIST 是一个简单的计算机视觉数据集，它是由一系列手写数字图片组成的.

![](images/data_sample.png)

在数据集中，每一张图片会有一个标签 label, 表示该张图片上的数字是什么。比如以上图片所对应的标签是：5,0,4,1

MNIST 由 70000 张手写数字 (0~9) 图片 (灰度图) 组成，由很多不同的人写成，其中 60000 张是训练集，另外 10000 张是测试集.

每张图片的大小是 28 x 28 像素，数字的大小是 20 x 20，位于图片的中心.

![](images/data_single.png)

MNIST 数据集可以在网站 http://yann.lecun.com/exdb/mnist/ 下载到.

一个图片可以被一个 1*784 的向量所表示, 下面将使用这种结构.


In [11]:
# 在 TensorFlow 中为封装了一个方法，只需要调用这个方法，程序就会自动去下载和获取数据集

from tensorflow.examples.tutorials.mnist import input_data

# 下载 (如果没有下载) 和读取 MNIST 的训练集、测试集和验证集
# 大小分别为：55000、10000、5000
# 经过编码之后，训练集的标签数据就变成了 [55000,10] 的浮点类型数组
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


## 2. Tensorflow 实现多分类模型：Softmax Regressions

使用 Softmax Regressions 将图片在这 10 个类中进行分类。

In [2]:
import tensorflow as tf

# 为输入数据 x 创建占位符
'''
这里的 x 并不是具体的数值，而是一个占位符
当 TensorFlow 运行计算的时候，再传入 x 的真实数据
我们的输入数据是 n 个 1*784 的向量，可以表示成 2 层的 tensor, 大小是 [None,784]
None 表示到时候传输的数据可以任何长度，也就是说可以是任何数量的样本点
'''
x = tf.placeholder(tf.float32, [None, 784])

# 创建一个线性模型，由权重 w，与偏置项 b 组成
# w 和 b 是在训练过程中不断改变不断优化的，使用 Variable 来创建
# w 的大小是 [784,10], 表示 784 个像素输入点乘以 10 维的向量（10 个类别）
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 建立 softmax 模型
# softmax 函数将 n 个非负的值归一化为 0~1 之间的值，形成一个概率分布
y = tf.nn.softmax(tf.matmul(x, W) + b)

## 3. 模型训练

### 3.1 实现交叉熵函数

交叉熵从一定意义上可以度量模型对于真实情况的拟合程度，交叉熵越大，则模型越不拟合，表现力越差。

In [3]:
# 为真实的标签添加占位符
y_ = tf.placeholder(tf.float32, [None, 10])

# 创建交叉熵函数
'''
tf.log 表示将 y 的每一个元素都做 log 运算；然后将其乘以对应的真实标签的类别 y_中的元素
tf.reduce_sum 表示的是将索引为 1 的值进行求和
tf.reduce_mean 表示对所有样本的交叉熵求均值
'''
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

### 3.2 使用 BP 算法优化参数

In [7]:
# 使用梯度下降法最小化交叉熵损失，学习率设置为 0.3
train_step = tf.train.GradientDescentOptimizer(0.3).minimize(cross_entropy)

### 3.3 运行迭代

模型训练的 graph 基本已经完成，现在可以初始化变量，创建会话，以进行循环训练。

In [9]:
# 创建会话
sess = tf.InteractiveSession()

# 初始化所有变量
tf.global_variables_initializer().run()

# 循环1000次训练模型
'''
在每次循环中，从训练集中获取 100 个样本, 然后运行 train_step 操作，并且给之前的占位符喂入数据
应用了随机梯度下降法。直接使用所有样本来做循环迭代的 “梯度下降法” 会大大增加计算的成本
而 “随机梯度下降法” 减少了计算量并且也保持了相对一致的准确率
'''
for _ in range(1000):
    
  # 获取训练集与标签集，每次获取100个样本
  batch_xs, batch_ys = mnist.train.next_batch(100)  
    
  # 喂数据，训练
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

## 4. 模型评估

In [10]:
# tf.argmax 返回的是一个 tensor 里在某个维度上最大值的索引（对应某个类别）
# tf.equal 返回的是一个布尔类型的列表
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# 求正确率，只要将布尔类型的列表全部求和再求均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 喂入测试数据集，来运行计算测试集上的准确率
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9171
